In [1]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
%matplotlib inline

df = pd.read_csv("salario.csv")
count = defaultdict(int)

In [2]:
def testing():
    return df.groupby(["pais","estado","cidade","genero","data"], axis=0, level=None, as_index=False).count()

def classEquivalence(a,b,c,d,e):
    return df[(df["pais"] == a) & (df["estado"] == b) & (df["cidade"] == c) & (df["genero"] == d) & (df["data"] == e)]
    

In [3]:
#def toGeneralize(aux):
#    maximum = -1
#    ret = 0
#    for j in df.columns.drop('salario').drop('id'):
#if (maximum < int(aux.groupby(j).count().shape[0])):
#            maximum = int(aux.groupby(j).count().shape[0])
#            ret = j

#    return ret
    
def toGeneralize2(genero): 
    if genero == True: random.choice(df.columns.drop('salario').drop('id'))
    return random.choice(df.columns.drop('salario').drop('id').drop('genero'))

def nextGeneralization(i,j, value):
    if(j == "pais"): 
        if(df.at[i,"estado"] != "*"): return nextGeneralization(i, "estado", value)
        df.at[i, j] = "*"
        if(count["lugar",i] == 2): count["lugar",i] = 3
    elif(j == "estado"):
        if(df.at[i,"cidade"] != "*"): return nextGeneralization(i, "cidade", value)
        df.at[i, j] = "*"
        if(count["lugar",i] == 1): count["lugar",i] = 2
    elif(j == "cidade"):
        df.at[i, j] = "*"
        if(count["lugar",i] == 0): count["lugar",i] = 1
    elif(j == "genero"):
        df.at[i, j] = "*"
        if(count["genero",i] == 0): count["genero",i] = 1
    elif(j == "data"):
        if(str(value)[0:5] == "**/**"): df.at[i, j] = "**/**/**"
        elif(str(value)[0:2] == "**"): df.at[i, j] = "**/**"  + str((value)[5:8])
        else:
            df.at[i, j] = "**" + str((value)[2:8])
        
        if(count["data",i] < 3): count["data",i] = count["data",i] + 1
    else:
        df.at[i, j] = "erro"
        

In [4]:
def max_hye(col):
    if (col == 'genero'): return 1
    return 3

Na = 3 #n_atributos semi identificadores
def precision():
    summ =  0
    for index, row in df.iterrows():
        for col in df.columns.drop('id').drop('salario'):
                summ += count[col, index]/max_hye(col)

    return 1 - summ/(df.shape[0] * Na)

In [5]:
def nextt(k, genero):
        
        test = testing()
        test = test[test['salario'] < k]
        
        for index,aux in test.iterrows():
            a = aux['pais']
            b = aux['estado']
            c = aux['cidade']
            d = aux['genero']
            e = aux['data']
            
            #if (random.choice([True,False]) and random.choice([True,False])):
            #    j = toGeneralize(classEquivalence(a,b,c,d,e))
            #else:
            j = toGeneralize2(genero)

            for i in classEquivalence(a,b,c,d,e).index:
                print(str(i) + " " + str(j))
                nextGeneralization(i, j, aux[j])
            print("----------------\n\n")

def kanonimity(k):
    while testing()[testing()['salario'] < k].shape[0] > 0:
        qtd_restante = testing()[testing()['salario'] < k].shape[0]
        print('@@@@@@@@@@@@@@@@')
        print(qtd_restante)
        print('@@@@@@@@@@@@@@@@')
        if(qtd_restante > 50): 
            nextt(k, False)
        else:
            nextt(k, True)

        

In [19]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(2)


@@@@@@@@@@@@@@@@
1000
@@@@@@@@@@@@@@@@
821 estado
----------------


543 data
----------------


273 estado
----------------


848 cidade
----------------


529 cidade
----------------


911 data
----------------


382 pais
----------------


694 estado
----------------


918 cidade
----------------


390 estado
----------------


849 data
----------------


240 cidade
----------------


475 estado
----------------


514 data
----------------


262 data
----------------


717 pais
----------------


50 pais
----------------


636 data
----------------


633 cidade
----------------


770 cidade
----------------


708 estado
----------------


481 cidade
----------------


83 data
----------------


79 pais
----------------


88 pais
----------------


502 pais
----------------


115 cidade
----------------


127 estado
----------------


224 pais
----------------


16 estado
----------------


782 cidade
----------------


827 estado
----------------


915 estado
----------------


623 

479 data
----------------


63 pais
----------------


156 cidade
----------------


337 pais
----------------


681 cidade
----------------


341 cidade
----------------


658 estado
----------------


152 estado
----------------


528 estado
----------------


688 data
----------------


345 data
----------------


982 estado
----------------


285 data
----------------


538 pais
----------------


999 pais
----------------


547 estado
----------------


616 pais
----------------


896 estado
----------------


966 data
----------------


789 pais
----------------


112 cidade
----------------


482 cidade
----------------


405 pais
----------------


685 estado
----------------


245 cidade
----------------


719 data
----------------


89 estado
----------------


663 data
----------------


541 pais
----------------


460 estado
----------------


874 pais
----------------


565 estado
----------------


764 estado
----------------


797 cidade
----------------


403 estado
---

550 data
----------------


542 cidade
----------------


470 pais
----------------


142 cidade
----------------


19 data
----------------


909 pais
----------------


440 estado
----------------


387 pais
----------------


145 cidade
----------------


553 cidade
----------------


562 cidade
----------------


884 estado
----------------


49 data
----------------


772 estado
----------------


741 cidade
----------------


790 data
----------------


879 data
----------------


948 pais
----------------


404 cidade
----------------


832 pais
----------------


595 data
----------------


707 cidade
----------------


666 cidade
----------------


986 pais
----------------


597 data
----------------


839 data
----------------


94 estado
----------------


594 estado
----------------


2 estado
----------------


830 pais
----------------


20 pais
----------------


418 cidade
----------------


241 cidade
----------------


590 estado
----------------


843 pais
---------

161 cidade
----------------


926 cidade
----------------


573 pais
----------------


416 estado
----------------


309 cidade
----------------


813 pais
----------------


424 cidade
----------------


67 data
----------------


932 data
----------------


487 pais
----------------


716 pais
----------------


699 cidade
----------------


396 cidade
----------------


963 pais
----------------


842 cidade
----------------


21 estado
----------------


984 pais
----------------


665 data
----------------


593 pais
----------------


554 pais
----------------


532 cidade
----------------


834 cidade
----------------


351 data
----------------


897 data
----------------


393 data
----------------


435 estado
----------------


744 estado
----------------


215 data
----------------


498 pais
----------------


960 data
----------------


951 data
----------------


901 data
----------------


222 estado
----------------


756 estado
----------------


478 cidade
---------

676 data
----------------


342 estado
----------------


327 cidade
----------------


857 cidade
----------------


265 cidade
----------------


415 pais
----------------


920 cidade
----------------


389 cidade
----------------


225 estado
----------------


997 pais
----------------


567 cidade
----------------


964 cidade
----------------


186 pais
----------------


885 data
----------------


213 data
----------------


949 data
----------------


704 estado
----------------


677 cidade
----------------


227 cidade
----------------


235 estado
----------------


344 data
----------------


702 cidade
----------------


68 pais
----------------


131 pais
----------------


391 cidade
----------------


865 estado
----------------


519 pais
----------------


178 estado
----------------


190 estado
----------------


439 data
----------------


85 data
----------------


47 estado
----------------


858 cidade
----------------


784 estado
----------------


518 estad

...

602 data
----------------


673 pais
----------------


425 cidade
----------------


14 pais
----------------


219 pais
----------------


397 data
----------------


451 data
----------------


829 estado
----------------


467 estado
----------------


73 cidade
----------------


973 estado
----------------


349 cidade
----------------


988 data
----------------


643 cidade
----------------


897 cidade
----------------


558 data
----------------


638 cidade
----------------


61 cidade
----------------


813 cidade
----------------


811 estado
----------------


592 cidade
----------------


798 cidade
----------------


255 pais
----------------


483 pais
----------------


738 cidade
----------------


744 pais
----------------


834 pais
----------------


252 data
----------------


17 data
----------------


264 data
----------------


548 pais
----------------


893 cidade
----------------


614 data
----------------


426 data
----------------


126 pais
-----------

608 pais
----------------


610 pais
----------------


99 cidade
----------------


459 pais
----------------


474 pais
----------------


1 estado
----------------


45 cidade
----------------


537 estado
----------------


866 pais
----------------


257 cidade
----------------


613 cidade
----------------


122 estado
----------------


155 data
----------------


450 cidade
----------------


584 data
----------------


472 cidade
----------------


384 cidade
----------------


525 estado
----------------


381 cidade
----------------


910 cidade
----------------


376 cidade
----------------


347 estado
----------------


505 data
----------------


729 estado
----------------


132 cidade
----------------


12 data
----------------


553 pais
----------------


33 pais
----------------


260 cidade
----------------


203 cidade
----------------


929 estado
----------------


418 data
----------------


160 data
----------------


116 estado
----------------


94 data
----

525 data
----------------


381 cidade
----------------


910 cidade
----------------


376 cidade
----------------


347 data
----------------


729 cidade
----------------


132 pais
----------------


260 cidade
----------------


203 cidade
----------------


929 cidade
----------------


452 estado
----------------


116 data
----------------


142 estado
----------------


504 data
----------------


606 pais
----------------


707 estado
----------------


239 cidade
----------------


49 estado
----------------


790 data
----------------


2 data
----------------


137 cidade
----------------


815 data
----------------


670 data
----------------


612 cidade
----------------


869 estado
----------------


696 data
----------------


602 pais
----------------


331 estado
----------------


893 pais
----------------


425 data
----------------


498 data
----------------


283 estado
----------------


349 estado
----------------


638 cidade
----------------


665 cidade
--

772 estado
----------------


560 estado
----------------


169 estado
----------------


64 pais
----------------


459 cidade
----------------


247 pais
----------------


704 pais
----------------


512 pais
----------------


161 pais
----------------


804 cidade
----------------


833 cidade
----------------


10 cidade
----------------


978 cidade
----------------


210 cidade
----------------


461 estado
----------------


268 estado
----------------


308 data
----------------


895 cidade
----------------


961 estado
----------------


677 cidade
----------------


261 pais
----------------


213 cidade
----------------


885 data
----------------


399 pais
----------------


541 estado
----------------


684 pais
----------------


122 pais
----------------


655 pais
----------------


409 estado
----------------


737 data
----------------


1 data
----------------


99 data
----------------


450 estado
----------------


260 data
----------------


381 estado
------

716 estado
----------------


444 cidade
----------------


579 data
----------------


374 estado
----------------


847 estado
----------------


877 estado
----------------


606 pais
----------------


918 estado
----------------


874 estado
----------------


935 cidade
----------------


292 data
----------------


456 data
----------------


491 pais
----------------


169 pais
----------------


704 pais
----------------


512 estado
----------------


161 estado
----------------


804 data
----------------


833 estado
----------------


210 data
----------------


961 cidade
----------------


677 estado
----------------


@@@@@@@@@@@@@@@@
30
@@@@@@@@@@@@@@@@
368 pais
----------------


392 pais
----------------


343 pais
----------------


959 pais
----------------


396 data
----------------


881 pais
----------------


521 pais
----------------


569 data
----------------


655 pais
----------------


908 pais
----------------


88 estado
----------------


829 estado
-

In [20]:
precision() #k =2

0.8546666666666678

In [8]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(4)


@@@@@@@@@@@@@@@@
1000
@@@@@@@@@@@@@@@@
821 pais
----------------


543 cidade
----------------


273 data
----------------


848 data
----------------


529 pais
----------------


911 data
----------------


382 estado
----------------


694 estado
----------------


918 cidade
----------------


390 data
----------------


849 estado
----------------


240 pais
----------------


475 pais
----------------


514 cidade
----------------


262 data
----------------


717 estado
----------------


50 cidade
----------------


636 cidade
----------------


633 pais
----------------


770 estado
----------------


708 pais
----------------


481 pais
----------------


83 cidade
----------------


79 data
----------------


88 cidade
----------------


502 data
----------------


115 estado
----------------


127 estado
----------------


224 cidade
----------------


16 data
----------------


782 cidade
----------------


827 pais
----------------


915 pais
----------------


623 data
-

688 data
----------------


345 pais
----------------


982 data
----------------


285 data
----------------


538 cidade
----------------


999 estado
----------------


547 pais
----------------


616 cidade
----------------


896 cidade
----------------


966 data
----------------


789 estado
----------------


112 data
----------------


482 data
----------------


405 estado
----------------


685 pais
----------------


245 data
----------------


719 estado
----------------


89 cidade
----------------


663 cidade
----------------


541 estado
----------------


460 cidade
----------------


874 estado
----------------


565 data
----------------


764 data
----------------


797 pais
----------------


403 data
----------------


105 cidade
----------------


572 estado
----------------


417 cidade
----------------


684 estado
----------------


564 data
----------------


169 cidade
----------------


647 data
----------------


690 data
----------------


157 pais
------

470 data
----------------


142 estado
----------------


19 pais
----------------


909 pais
----------------


440 pais
----------------


387 estado
----------------


145 cidade
----------------


553 estado
----------------


562 estado
----------------


884 pais
----------------


49 estado
----------------


772 pais
----------------


741 pais
----------------


790 cidade
----------------


879 data
----------------


948 estado
----------------


404 data
----------------


832 pais
----------------


595 cidade
----------------


707 cidade
----------------


666 estado
----------------


986 pais
----------------


597 data
----------------


839 estado
----------------


94 data
----------------


594 data
----------------


2 pais
----------------


830 pais
----------------


20 pais
----------------


418 data
----------------


241 cidade
----------------


590 data
----------------


843 cidade
----------------


137 cidade
----------------


154 data
---------------

181 data
----------------


161 estado
----------------


926 pais
----------------


573 estado
----------------


416 data
----------------


309 cidade
----------------


813 estado
----------------


424 pais
----------------


67 pais
----------------


932 pais
----------------


487 pais
----------------


716 data
----------------


699 pais
----------------


396 pais
----------------


963 pais
----------------


842 data
----------------


21 cidade
----------------


984 data
----------------


665 cidade
----------------


593 cidade
----------------


554 cidade
----------------


532 estado
----------------


834 pais
----------------


351 data
----------------


897 estado
----------------


393 pais
----------------


435 data
----------------


744 data
----------------


215 estado
----------------


498 estado
----------------


960 pais
----------------


951 cidade
----------------


901 estado
----------------


222 pais
----------------


756 estado
-----------

997 pais
----------------


567 data
----------------


391 data
----------------


964 data
----------------


186 estado
----------------


518 pais
----------------


885 data
----------------


677 cidade
----------------


865 estado
----------------


515 data
----------------


235 estado
----------------


344 pais
----------------


441 estado
----------------


342 pais
----------------


131 pais
----------------


778 data
----------------


178 cidade
----------------


227 data
----------------


959 pais
----------------


702 pais
----------------


704 data
----------------


190 data
----------------


503 data
----------------


949 cidade
----------------


857 pais
----------------


327 cidade
----------------


85 data
----------------


346 data
----------------


389 pais
----------------


784 cidade
----------------


213 pais
----------------


259 cidade
----------------


878 data
----------------


156 estado
----------------


63 estado
----------------


...

909 estado
----------------


304 data
401 data
772 data
----------------


34 cidade
562 cidade
714 cidade
----------------


199 data
----------------


452 estado
----------------


185 estado
----------------


728 estado
----------------


725 estado
----------------


288 pais
----------------


142 pais
----------------


325 cidade
----------------


49 data
----------------


20 pais
----------------


590 data
----------------


241 cidade
----------------


137 cidade
----------------


815 pais
----------------


606 pais
----------------


560 pais
----------------


767 estado
----------------


11 estado
----------------


752 data
----------------


670 estado
----------------


785 estado
----------------


272 cidade
----------------


644 data
----------------


833 estado
----------------


860 cidade
----------------


357 data
416 data
----------------


368 data
829 data
----------------


189 estado
973 estado
----------------


712 data
----------------


277 d

155 cidade
----------------


550 estado
----------------


596 pais
----------------


646 cidade
----------------


994 data
----------------


512 pais
----------------


448 estado
----------------


983 cidade
----------------


458 estado
----------------


422 data
----------------


485 estado
----------------


46 pais
----------------


311 pais
----------------


698 data
----------------


891 cidade
----------------


369 cidade
450 cidade
----------------


796 pais
----------------


291 data
980 data
----------------


193 data
----------------


542 estado
----------------


20 data
----------------


384 estado
----------------


815 cidade
----------------


595 cidade
703 cidade
----------------


33 cidade
----------------


884 data
----------------


175 pais
----------------


786 pais
----------------


320 cidade
----------------


132 estado
----------------


270 pais
----------------


832 pais
----------------


510 data
----------------


43 estado
------

@@@@@@@@@@@@@@@@
156
@@@@@@@@@@@@@@@@
466 pais
----------------


586 cidade
----------------


375 pais
----------------


705 data
----------------


654 cidade
----------------


356 pais
----------------


516 pais
----------------


581 cidade
----------------


72 data
645 data
----------------


396 estado
----------------


701 pais
----------------


881 data
----------------


425 estado
----------------


600 estado
----------------


637 cidade
----------------


280 pais
----------------


555 pais
----------------


919 estado
----------------


837 cidade
----------------


844 pais
----------------


925 data
----------------


655 data
----------------


806 data
----------------


270 estado
----------------


445 data
----------------


621 cidade
----------------


975 pais
----------------


882 data
----------------


832 data
----------------


12 pais
----------------


653 estado
----------------


219 pais
----------------


152 estado
284 estado
-------------

226 pais
----------------


683 pais
725 pais
----------------


363 pais
494 pais
860 pais
----------------


756 estado
933 estado
----------------


84 pais
----------------


69 cidade
----------------


141 pais
----------------


617 data
----------------


903 estado
----------------


196 estado
----------------


608 cidade
----------------


387 estado
----------------


218 cidade
----------------


460 estado
----------------


481 estado
----------------


898 pais
----------------


628 estado
----------------


596 estado
----------------


169 pais
----------------


264 cidade
----------------


790 data
----------------


8 cidade
----------------


161 data
----------------


951 pais
----------------


827 data
----------------


330 estado
----------------


455 pais
----------------


944 cidade
----------------


345 data
----------------


407 estado
----------------


572 cidade
----------------


574 pais
847 pais
----------------


106 cidade
----------------

701 data
----------------


84 data
----------------


155 data
----------------


Wall time: 18.3 s


In [9]:
precision() #k=4

0.8255555555555559

In [10]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(8)


@@@@@@@@@@@@@@@@
1000
@@@@@@@@@@@@@@@@
821 estado
----------------


543 pais
----------------


273 pais
----------------


848 pais
----------------


529 cidade
----------------


911 data
----------------


382 data
----------------


694 data
----------------


918 data
----------------


390 estado
----------------


849 cidade
----------------


240 pais
----------------


475 data
----------------


514 cidade
----------------


262 cidade
----------------


717 cidade
----------------


50 cidade
----------------


636 cidade
----------------


633 pais
----------------


770 estado
----------------


708 data
----------------


481 data
----------------


83 pais
----------------


79 data
----------------


88 estado
----------------


502 estado
----------------


115 pais
----------------


127 estado
----------------


224 pais
----------------


16 cidade
----------------


782 estado
----------------


827 pais
----------------


915 estado
----------------


623 data
-

63 cidade
----------------


156 pais
----------------


337 cidade
----------------


681 data
----------------


341 estado
----------------


658 pais
----------------


152 data
----------------


528 cidade
----------------


688 cidade
----------------


345 estado
----------------


982 estado
----------------


285 data
----------------


538 pais
----------------


999 pais
----------------


547 estado
----------------


616 pais
----------------


896 estado
----------------


966 cidade
----------------


789 pais
----------------


112 pais
----------------


482 cidade
----------------


405 pais
----------------


685 estado
----------------


245 cidade
----------------


719 estado
----------------


89 estado
----------------


663 pais
----------------


541 data
----------------


460 cidade
----------------


874 data
----------------


565 data
----------------


764 pais
----------------


797 pais
----------------


403 pais
----------------


105 pais
---------

291 cidade
----------------


635 data
----------------


533 cidade
----------------


771 cidade
----------------


280 data
----------------


550 cidade
----------------


542 pais
----------------


470 cidade
----------------


142 pais
----------------


19 cidade
----------------


909 estado
----------------


440 cidade
----------------


387 cidade
----------------


145 pais
----------------


553 cidade
----------------


562 cidade
----------------


884 pais
----------------


49 estado
----------------


772 pais
----------------


741 cidade
----------------


790 data
----------------


879 estado
----------------


948 pais
----------------


404 data
----------------


832 cidade
----------------


595 data
----------------


707 cidade
----------------


666 estado
----------------


986 data
----------------


597 pais
----------------


839 cidade
----------------


94 pais
----------------


594 cidade
----------------


2 data
----------------


830 estado
----

275 pais
----------------


8 estado
----------------


368 pais
----------------


73 estado
----------------


181 pais
----------------


161 pais
----------------


926 estado
----------------


573 data
----------------


416 cidade
----------------


309 estado
----------------


813 pais
----------------


424 cidade
----------------


67 pais
----------------


932 data
----------------


487 data
----------------


716 pais
----------------


699 estado
----------------


396 data
----------------


963 estado
----------------


842 estado
----------------


21 estado
----------------


984 cidade
----------------


665 cidade
----------------


593 pais
----------------


554 pais
----------------


532 pais
----------------


834 estado
----------------


351 estado
----------------


897 estado
----------------


393 estado
----------------


435 pais
----------------


744 data
----------------


215 estado
----------------


498 estado
----------------


960 data
--------

778 cidade
----------------


111 data
----------------


599 data
----------------


259 cidade
----------------


858 pais
----------------


570 data
----------------


503 pais
----------------


784 data
----------------


676 pais
----------------


85 estado
----------------


857 data
----------------


265 pais
----------------


415 data
----------------


920 estado
----------------


389 cidade
----------------


131 estado
----------------


225 cidade
----------------


997 cidade
----------------


567 cidade
----------------


391 data
----------------


964 data
----------------


186 cidade
----------------


518 cidade
----------------


885 data
----------------


178 data
----------------


213 estado
----------------


949 pais
----------------


677 estado
----------------


227 cidade
----------------


865 estado
----------------


515 pais
----------------


235 cidade
----------------


344 pais
----------------


702 data
----------------


441 data
--------

...

260 pais
525 pais
836 pais
----------------


29 estado
----------------


929 pais
----------------


13 data
----------------


263 estado
270 estado
443 estado
531 estado
590 estado
823 estado
838 estado
----------------


809 estado
----------------


597 cidade
----------------


852 data
----------------


109 cidade
----------------


477 estado
----------------


752 cidade
----------------


754 estado
----------------


579 pais
----------------


304 pais
711 pais
735 pais
816 pais
968 pais
----------------


142 cidade
239 cidade
790 cidade
840 cidade
894 cidade
----------------


491 pais
----------------


279 data
642 data
----------------


182 estado
----------------


196 pais
628 pais
877 pais
----------------


452 pais
----------------


387 cidade
----------------


767 data
909 data
----------------


453 data
----------------


713 pais
----------------


324 data
----------------


606 data
----------------


787 data
----------------


669 pais
---------------

55 estado
399 estado
565 estado
----------------


345 estado
684 estado
709 estado
----------------


63 pais
410 pais
999 pais
----------------


479 estado
----------------


690 estado
----------------


112 estado
495 estado
896 estado
----------------


572 pais
----------------


457 data
----------------


691 pais
----------------


541 data
874 data
----------------


105 pais
114 pais
169 pais
337 pais
563 pais
----------------


791 estado
----------------


7 estado
----------------


71 data
278 data
----------------


899 cidade
----------------


460 data
----------------


547 estado
----------------


141 cidade
----------------


538 data
----------------


528 data
----------------


285 data
----------------


198 estado
----------------


52 cidade
193 cidade
577 cidade
621 cidade
768 cidade
980 cidade
----------------


118 data
----------------


44 cidade
----------------


321 estado
----------------


35 pais
----------------


432 estado
----------------


6

82 data
----------------


99 data
----------------


459 data
----------------


669 pais
----------------


53 pais
----------------


295 cidade
----------------


290 data
----------------


184 data
----------------


273 data
----------------


992 data
----------------


226 estado
----------------


951 cidade
----------------


426 pais
----------------


102 pais
----------------


760 estado
----------------


252 data
----------------


344 data
----------------


210 pais
----------------


717 data
----------------


195 data
----------------


810 data
----------------


113 estado
390 estado
527 estado
604 estado
----------------


127 estado
240 estado
620 estado
821 estado
848 estado
868 estado
----------------


898 data
----------------


115 cidade
693 cidade
----------------


529 pais
----------------


268 data
633 data
----------------


293 estado
----------------


78 data
----------------


514 cidade
543 cidade
----------------


6 estado
294 estado
619 est

430 cidade
792 cidade
----------------


119 data
373 data
537 data
635 data
765 data
----------------


486 estado
----------------


513 cidade
----------------


450 estado
----------------


109 pais
----------------


29 estado
754 estado
----------------


263 estado
270 estado
443 estado
531 estado
590 estado
823 estado
838 estado
----------------


477 data
----------------


597 estado
----------------


752 data
----------------


353 pais
787 pais
----------------


182 cidade
----------------


324 data
387 data
----------------


705 pais
----------------


244 estado
802 estado
904 estado
----------------


607 cidade
----------------


14 estado
517 estado
----------------


941 estado
----------------


558 estado
----------------


197 estado
----------------


5 cidade
----------------


926 estado
----------------


@@@@@@@@@@@@@@@@
114
@@@@@@@@@@@@@@@@
179 pais
----------------


343 cidade
444 cidade
712 cidade
789 cidade
----------------


44 data
----------------

@@@@@@@@@@@@@@@@
46
@@@@@@@@@@@@@@@@
189 data
973 data
----------------


343 pais
384 pais
444 pais
712 pais
789 pais
----------------


173 data
465 data
734 data
845 data
955 data
----------------


19 data
----------------


42 cidade
112 cidade
371 cidade
495 cidade
896 cidade
913 cidade
982 cidade
----------------


360 estado
365 estado
554 estado
668 estado
778 estado
----------------


331 cidade
691 cidade
749 cidade
799 cidade
956 cidade
----------------


335 data
----------------


678 pais
----------------


338 cidade
----------------


533 estado
----------------


521 cidade
----------------


299 cidade
----------------


644 data
----------------


758 data
----------------


801 data
----------------


212 data
----------------


861 data
----------------


73 data
----------------


162 estado
----------------


213 data
616 data
727 data
----------------


102 data
----------------


217 estado
698 estado
775 estado
----------------


78 data
760 data
------------

In [11]:
precision() #k=8

0.8052222222222221

In [12]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(16)


@@@@@@@@@@@@@@@@
1000
@@@@@@@@@@@@@@@@
821 data
----------------


543 data
----------------


273 estado
----------------


848 pais
----------------


529 data
----------------


911 data
----------------


382 estado
----------------


694 cidade
----------------


918 pais
----------------


390 cidade
----------------


849 data
----------------


240 cidade
----------------


475 pais
----------------


514 estado
----------------


262 pais
----------------


717 cidade
----------------


50 pais
----------------


636 cidade
----------------


633 cidade
----------------


770 estado
----------------


708 estado
----------------


481 estado
----------------


83 pais
----------------


79 cidade
----------------


88 data
----------------


502 data
----------------


115 cidade
----------------


127 estado
----------------


224 estado
----------------


16 data
----------------


782 estado
----------------


827 pais
----------------


915 cidade
----------------


623 es

479 estado
----------------


63 data
----------------


156 pais
----------------


337 estado
----------------


681 estado
----------------


341 cidade
----------------


658 cidade
----------------


152 cidade
----------------


528 cidade
----------------


688 data
----------------


345 data
----------------


982 data
----------------


285 cidade
----------------


538 data
----------------


999 estado
----------------


547 data
----------------


616 estado
----------------


896 estado
----------------


966 cidade
----------------


789 data
----------------


112 estado
----------------


482 data
----------------


405 data
----------------


685 pais
----------------


245 estado
----------------


719 data
----------------


89 pais
----------------


663 cidade
----------------


541 cidade
----------------


460 data
----------------


874 estado
----------------


565 data
----------------


764 pais
----------------


797 pais
----------------


403 cidade
-----

980 estado
----------------


291 pais
----------------


635 cidade
----------------


533 data
----------------


771 data
----------------


280 estado
----------------


550 data
----------------


542 estado
----------------


470 cidade
----------------


142 data
----------------


19 data
----------------


909 pais
----------------


440 pais
----------------


387 estado
----------------


145 estado
----------------


553 data
----------------


562 cidade
----------------


884 pais
----------------


49 estado
----------------


772 estado
----------------


741 data
----------------


790 data
----------------


879 cidade
----------------


948 estado
----------------


404 cidade
----------------


832 pais
----------------


595 data
----------------


707 pais
----------------


666 cidade
----------------


986 pais
----------------


597 data
----------------


839 pais
----------------


94 cidade
----------------


594 data
----------------


2 pais
--------------

540 data
----------------


124 cidade
----------------


194 data
----------------


912 cidade
----------------


275 estado
----------------


8 cidade
----------------


368 pais
----------------


73 data
----------------


181 cidade
----------------


161 pais
----------------


926 cidade
----------------


573 estado
----------------


416 data
----------------


309 cidade
----------------


813 estado
----------------


424 estado
----------------


67 data
----------------


932 pais
----------------


487 cidade
----------------


716 pais
----------------


699 data
----------------


396 data
----------------


963 cidade
----------------


842 pais
----------------


21 pais
----------------


984 data
----------------


665 estado
----------------


593 cidade
----------------


554 data
----------------


532 cidade
----------------


834 estado
----------------


351 pais
----------------


897 cidade
----------------


393 data
----------------


435 estado
--------

346 estado
----------------


749 estado
----------------


778 cidade
----------------


111 pais
----------------


599 data
----------------


259 estado
----------------


971 cidade
----------------


575 cidade
----------------


503 pais
----------------


784 cidade
----------------


676 cidade
----------------


85 cidade
----------------


327 pais
----------------


265 cidade
----------------


415 estado
----------------


920 cidade
----------------


389 pais
----------------


131 data
----------------


225 data
----------------


567 pais
----------------


964 cidade
----------------


186 pais
----------------


518 estado
----------------


885 estado
----------------


178 cidade
----------------


213 pais
----------------


704 cidade
----------------


227 cidade
----------------


865 estado
----------------


515 cidade
----------------


235 cidade
----------------


702 estado
----------------


342 cidade
----------------


68 estado
----------------


99

...

251 data
----------------


8 cidade
738 cidade
951 cidade
----------------


700 pais
----------------


558 estado
----------------


281 data
----------------


643 pais
----------------


607 pais
----------------


478 estado
----------------


@@@@@@@@@@@@@@@@
187
@@@@@@@@@@@@@@@@
32 cidade
40 cidade
52 cidade
118 cidade
130 cidade
164 cidade
333 cidade
428 cidade
556 cidade
572 cidade
733 cidade
844 cidade
956 cidade
----------------


100 data
376 data
464 data
632 data
829 data
852 data
927 data
991 data
----------------


79 cidade
134 cidade
179 cidade
272 cidade
----------------


193 data
406 data
523 data
936 data
----------------


43 cidade
96 cidade
98 cidade
210 cidade
243 cidade
267 cidade
631 cidade
692 cidade
722 cidade
839 cidade
895 cidade
----------------


253 cidade
466 cidade
734 cidade
940 cidade
----------------


37 cidade
260 cidade
320 cidade
378 cidade
420 cidade
490 cidade
500 cidade
618 cidade
836 cidade
908 cidade
919 cidade
----------------


312 ci

321 pais
----------------


571 data
----------------


165 data
----------------


354 cidade
----------------


955 pais
----------------


653 estado
----------------


985 estado
----------------


211 pais
----------------


624 cidade
----------------


980 data
----------------


749 estado
----------------


826 cidade
----------------


666 data
----------------


50 cidade
----------------


761 estado
----------------


814 estado
----------------


89 cidade
----------------


222 cidade
----------------


13 pais
----------------


21 pais
713 pais
----------------


178 pais
325 pais
770 pais
972 pais
----------------


70 cidade
583 cidade
872 cidade
----------------


11 estado
81 estado
199 estado
213 estado
235 estado
394 estado
442 estado
468 estado
504 estado
616 estado
672 estado
843 estado
898 estado
981 estado
----------------


279 estado
430 estado
----------------


22 cidade
28 cidade
119 cidade
328 cidade
353 cidade
481 cidade
483 cidade
574 cidade
635 cidad

@@@@@@@@@@@@@@@@
90
@@@@@@@@@@@@@@@@
165 data
368 data
571 data
666 data
----------------


79 cidade
134 cidade
179 cidade
272 cidade
575 cidade
----------------


600 estado
712 estado
----------------


988 pais
----------------


43 data
96 data
98 data
210 data
243 data
267 data
321 data
631 data
692 data
722 data
802 data
839 data
895 data
915 data
----------------


195 estado
619 estado
649 estado
980 estado
----------------


37 cidade
251 cidade
260 cidade
320 cidade
378 cidade
420 cidade
490 cidade
500 cidade
618 cidade
836 cidade
908 cidade
919 cidade
----------------


312 cidade
380 cidade
432 cidade
524 cidade
634 cidade
645 cidade
703 cidade
----------------


42 pais
302 pais
----------------


511 cidade
603 cidade
624 cidade
----------------


754 cidade
819 cidade
----------------


117 estado
----------------


392 pais
584 pais
674 pais
929 pais
----------------


89 cidade
408 cidade
684 cidade
----------------


778 data
----------------


749 pais
-------------

525 estado
----------------


101 cidade
----------------


50 pais
511 pais
603 pais
624 pais
694 pais
826 pais
943 pais
----------------


754 pais
819 pais
----------------


89 pais
408 pais
684 pais
985 pais
----------------


346 estado
----------------


827 data
----------------


544 pais
----------------


531 pais
----------------


5 pais
----------------


354 pais
----------------


955 estado
----------------


211 cidade
----------------


222 cidade
----------------


13 pais
----------------


21 estado
527 estado
713 estado
----------------


248 pais
698 pais
----------------


166 pais
----------------


103 estado
----------------


174 estado
261 estado
268 estado
282 estado
453 estado
795 estado
863 estado
----------------


56 data
307 data
310 data
626 data
662 data
700 data
736 data
777 data
885 data
----------------


860 estado
----------------


744 cidade
875 cidade
----------------


558 estado
773 estado
909 estado
----------------


148 cidade
--------

115 cidade
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 pais
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 estado
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 estado
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 pais
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 estado
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 pais
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 cidade
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 pais
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 pais
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 estado
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 estado
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 estado
----------------


@@@@@@@@@@@@@@@@
1
@@@@@@@@@@@@@@@@
115 data
----------------


Wall time: 20.7 s


In [13]:
precision() #k=16

0.76788888888889

In [14]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(64)

@@@@@@@@@@@@@@@@
1000
@@@@@@@@@@@@@@@@
821 pais
----------------


543 pais
----------------


273 pais
----------------


848 cidade
----------------


529 estado
----------------


911 data
----------------


382 cidade
----------------


694 pais
----------------


918 pais
----------------


390 pais
----------------


849 data
----------------


240 data
----------------


475 pais
----------------


514 data
----------------


262 cidade
----------------


717 data
----------------


50 cidade
----------------


636 data
----------------


633 cidade
----------------


770 pais
----------------


708 data
----------------


481 estado
----------------


83 cidade
----------------


79 estado
----------------


88 cidade
----------------


502 cidade
----------------


115 data
----------------


127 cidade
----------------


224 data
----------------


16 pais
----------------


782 data
----------------


827 data
----------------


915 data
----------------


623 pais
---------

479 pais
----------------


63 estado
----------------


156 cidade
----------------


337 estado
----------------


681 pais
----------------


341 estado
----------------


658 pais
----------------


152 estado
----------------


528 data
----------------


688 cidade
----------------


345 data
----------------


982 pais
----------------


285 data
----------------


538 cidade
----------------


999 estado
----------------


547 pais
----------------


616 pais
----------------


896 data
----------------


966 pais
----------------


789 pais
----------------


112 cidade
----------------


482 pais
----------------


405 cidade
----------------


685 data
----------------


245 pais
----------------


719 pais
----------------


89 estado
----------------


663 data
----------------


541 estado
----------------


460 estado
----------------


874 cidade
----------------


565 pais
----------------


764 cidade
----------------


797 pais
----------------


403 cidade
---------

724 estado
----------------


93 data
----------------


148 cidade
----------------


76 data
----------------


193 data
----------------


608 cidade
----------------


980 data
----------------


291 data
----------------


635 estado
----------------


533 estado
----------------


771 cidade
----------------


280 cidade
----------------


550 cidade
----------------


542 estado
----------------


470 cidade
----------------


142 estado
----------------


19 pais
----------------


909 cidade
----------------


440 estado
----------------


387 cidade
----------------


145 data
----------------


553 estado
----------------


562 estado
----------------


884 cidade
----------------


49 estado
----------------


772 estado
----------------


741 cidade
----------------


790 estado
----------------


879 data
----------------


948 data
----------------


404 data
----------------


832 data
----------------


595 pais
----------------


707 pais
----------------


666 estado

350 data
----------------


592 data
----------------


5 cidade
----------------


310 estado
----------------


540 pais
----------------


124 estado
----------------


194 cidade
----------------


912 pais
----------------


275 cidade
----------------


8 estado
----------------


368 estado
----------------


73 cidade
----------------


181 pais
----------------


161 cidade
----------------


926 estado
----------------


573 data
----------------


416 data
----------------


309 data
----------------


813 data
----------------


424 cidade
----------------


67 cidade
----------------


932 estado
----------------


487 pais
----------------


716 estado
----------------


699 cidade
----------------


396 data
----------------


963 cidade
----------------


842 data
----------------


21 pais
----------------


984 data
----------------


665 estado
----------------


593 estado
----------------


554 pais
----------------


532 estado
----------------


834 estado
------

652 data
----------------


876 data
----------------


957 estado
----------------


956 estado
----------------


519 pais
----------------


944 data
----------------


734 data
----------------


749 estado
----------------


111 cidade
----------------


599 data
----------------


259 pais
----------------


858 cidade
----------------


570 data
----------------


971 cidade
----------------


575 data
----------------


503 data
----------------


784 pais
----------------


676 estado
----------------


342 pais
----------------


85 pais
----------------


857 cidade
----------------


415 pais
----------------


920 cidade
----------------


389 cidade
----------------


131 pais
----------------


997 estado
----------------


964 cidade
----------------


186 pais
----------------


518 estado
----------------


885 cidade
----------------


178 cidade
----------------


213 cidade
----------------


949 cidade
----------------


227 data
----------------


865 estado
----

...

848 estado
----------------


257 pais
----------------


662 cidade
----------------


909 cidade
----------------


765 data
----------------


329 pais
----------------


585 cidade
----------------


241 pais
----------------


560 cidade
----------------


142 pais
----------------


178 estado
----------------


745 pais
----------------


332 estado
----------------


424 estado
----------------


169 data
----------------


91 estado
----------------


545 pais
----------------


965 estado
----------------


512 pais
----------------


992 pais
----------------


230 cidade
461 cidade
882 cidade
993 cidade
----------------


623 data
----------------


408 pais
659 pais
----------------


177 estado
248 estado
268 estado
273 estado
286 estado
296 estado
308 estado
313 estado
363 estado
463 estado
617 estado
620 estado
770 estado
868 estado
918 estado
----------------


529 data
----------------


455 data
----------------


633 cidade
----------------


4 data
90 data
100 data

241 estado
----------------


560 estado
----------------


142 pais
----------------


178 data
----------------


745 data
----------------


332 estado
----------------


424 estado
----------------


91 cidade
----------------


545 estado
----------------


965 estado
----------------


512 data
----------------


992 estado
----------------


194 data
----------------


230 estado
461 estado
882 estado
993 estado
----------------


623 pais
----------------


177 cidade
248 cidade
268 cidade
273 cidade
286 cidade
296 cidade
308 cidade
313 cidade
363 cidade
463 cidade
617 cidade
620 cidade
770 cidade
868 cidade
918 cidade
----------------


455 cidade
529 cidade
----------------


633 data
----------------


4 estado
90 estado
100 estado
140 estado
200 estado
205 estado
259 estado
294 estado
305 estado
317 estado
339 estado
346 estado
400 estado
402 estado
439 estado
488 estado
503 estado
519 estado
521 estado
571 estado
600 estado
630 estado
668 estado
687 estado
721 estado
734 e

@@@@@@@@@@@@@@@@
93
@@@@@@@@@@@@@@@@
785 estado
----------------


98 estado
----------------


581 estado
----------------


500 estado
----------------


72 pais
101 pais
543 pais
637 pais
645 pais
703 pais
844 pais
----------------


678 cidade
----------------


526 data
----------------


80 pais
929 pais
----------------


408 estado
446 estado
659 estado
859 estado
957 estado
----------------


263 data
930 data
----------------


83 data
137 data
214 data
232 data
365 data
441 data
477 data
530 data
554 data
670 data
742 data
778 data
959 data
----------------


73 data
956 data
----------------


229 cidade
----------------


701 estado
----------------


208 pais
----------------


674 pais
----------------


751 estado
----------------


476 estado
----------------


908 data
----------------


651 estado
----------------


180 cidade
----------------


979 data
----------------


749 estado
----------------


207 pais
----------------


889 estado
----------------


797 dat

@@@@@@@@@@@@@@@@
52
@@@@@@@@@@@@@@@@
98 cidade
----------------


581 data
979 data
----------------


500 data
908 data
----------------


678 data
----------------


208 estado
----------------


701 data
----------------


674 cidade
----------------


38 pais
408 pais
446 pais
589 pais
659 pais
859 pais
957 pais
----------------


651 data
995 data
----------------


476 estado
----------------


229 cidade
----------------


751 cidade
----------------


769 cidade
----------------


180 estado
----------------


749 pais
----------------


207 estado
----------------


889 estado
----------------


679 pais
----------------


360 pais
----------------


102 data
121 data
247 data
258 data
279 data
415 data
566 data
642 data
909 data
----------------


241 data
----------------


745 estado
----------------


22 data
45 data
328 data
342 data
483 data
537 data
560 data
574 data
765 data
----------------


366 pais
----------------


848 pais
----------------


36 pais
239 pais
278

679 estado
----------------


180 cidade
----------------


207 pais
----------------


229 estado
889 estado
----------------


751 estado
----------------


360 data
476 data
----------------


749 pais
----------------


769 estado
----------------


36 pais
48 pais
61 pais
99 pais
142 pais
239 pais
265 pais
278 pais
288 pais
323 pais
344 pais
366 pais
377 pais
494 pais
513 pais
545 pais
546 pais
578 pais
622 pais
633 pais
656 pais
657 pais
710 pais
776 pais
807 pais
848 pais
897 pais
941 pais
----------------


91 pais
----------------


965 data
----------------


658 cidade
----------------


561 cidade
----------------


372 cidade
474 cidade
----------------


332 pais
----------------


585 cidade
992 cidade
----------------


792 data
----------------


177 data
248 data
268 data
273 data
286 data
296 data
308 data
313 data
363 data
463 data
617 data
620 data
770 data
868 data
918 data
----------------


152 cidade
340 cidade
341 cidade
345 cidade
410 cidade
457 cidade
495 ci

In [16]:
precision() #k=64

0.7344444444444465